In [1]:
import os,sys
import pandas as pd
import pickle
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from wer import calculate_wer

from Models import cnn_rnn_model,simple_rnn,model_2,bidirectional_rnn_model_gpu,bidirectional_rnn_model
from Batch import Batch


In [2]:
data=pd.read_json('../artifacts/meta.json')
data.drop(data[data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/features.pkl','rb') as tel:
    features=pickle.load(tel)
    tel.close()

In [3]:
valid_data=pd.read_json("../artifacts/valid_meta.json")
valid_data.drop(valid_data[valid_data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/valid_features.pkl','rb') as val:
    valid_features=pickle.load(val)
    val.close()

In [5]:
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()
MINI_BATCH_SIZE=250


audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [6]:
bidirectional_rnn=bidirectional_rnn_model_gpu(input_dim=26, 
                        units=250)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 500)         417000    
_________________________________________________________________
norm_bidir_rnn (BatchNormali (None, None, 500)         2000      
_________________________________________________________________
time_distributed (TimeDistri (None, None, 29)          14529     
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0         
Total params: 433,529
Trainable params: 432,529
Non-trainable params: 1,000
_________________________________________________________________
None


In [7]:
cnn_rnn = cnn_rnn_model(input_dim=26, # change to 13 if you would like to use MFCC features
                        filters=250,
                        kernel_size=4, 
                        conv_stride=1,
                        conv_border_mode='same',
                        units=200,output_dim=29)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 250)         26250     
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 250)         1000      
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         90200     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 200)         800       
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 29)          5829      
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0   

In [5]:
deep_speech = model_2(input_dim=26,
                filters=100,
                kernel_size=4, 
                conv_stride=1,
                conv_border_mode='valid',
                units=250,
                activation='tanh',
                dropout_rate=0.2,
                number_of_layers=2,
                output_dim=29)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
layer_1_conv (Conv1D)        (None, None, 100)         10500     
_________________________________________________________________
conv_batch_norm (BatchNormal (None, None, 100)         400       
_________________________________________________________________
rnn_1 (GRU)                  (None, None, 250)         264000    
_________________________________________________________________
bt_rnn_1 (BatchNormalization (None, None, 250)         1000      
_________________________________________________________________
final_layer_of_rnn (GRU)     (None, None, 250)         376500    
_________________________________________________________________
bt_rnn_final (BatchNormaliza (None, None, 250)         1000  

In [9]:
with open('models\\bidirectional_rnn_gpu.h5', 'rb') as t:
        bidirectional_rnn_model_hist=pickle.load( t)

In [10]:
with open('models\\deep_speech.pickle', 'rb') as f:
        deep_speech_hist=pickle.load( f)

In [11]:
with open('models\\simple_rnn_model.pickle', 'rb') as g:
        cnn_hist=pickle.load( g)

In [12]:
with open('models\cnn_rnn.h5', 'rb') as g:
        cnn_hist=pickle.load( g)

In [13]:
cnn_rnn.load_weights('./model/cnn.h5')

In [6]:
deep_speech.load_weights('./model/deep.h5')

In [15]:
from wer import predict

In [28]:
predict(audio_gen,4,'validation',deep_speech)

Truth: wanaeleza kupoteza maisha baada ya kutokea kwa mlipuko mwingine wa gesi
Predicted: waleleta kuputeka maishaba ya kutokea kwa mipokumenjini wajiefi
wer: 20


20

In [27]:
predict(audio_gen,4,'validation',cnn_rnn)

Truth: wanaeleza kupoteza maisha baada ya kutokea kwa mlipuko mwingine wa gesi
Predicted: wanelewa kuotena maisha ba ya kutoea wamipoungini wapi
wer: 22


22

In [18]:
predict(audio_gen,17,'validation',deep_speech)

Truth: ulitokea mapema siku ya ijumaa
Predicted: ulitoteamapemashikulevyuma
wer: 11


11

In [19]:
wer=

SyntaxError: invalid syntax (<ipython-input-19-dbaf8c1a3d06>, line 1)

In [7]:
import importlib 
import wer2
importlib.reload(wer2)
from wer2 import calculate_wer_entire,make_predictions

In [16]:
from tensorflow.keras.models import load_model,save_model

In [19]:
deep2=load_model('deep_speech_model_l.h5')

In [18]:
save_model(deep_speech,"deep_speech_model_l.h5")

In [23]:
make_predictions(deep_speech,[valid_features[0]])

['barendali yvemekoter we hedia koreakuspili kueiparili kwa a datikadini']

In [ ]:
calculate_wer_entire(deep_speech,valid_features,valid_data["text"].tolist())

0.9102984399728691

In [ ]:
calculate_wer_entire(cnn_rnn,valid_features,valid_data["text"].tolist())

0.9691951164368076